In [ ]:
!nvidia-smi

Sat Sep  4 13:39:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/Team-M1/badwords-classifier-train -b BM # 자기 브랜치 이름으로 변경
%cd badwords-classifier-train
!pip install -r requirements.txt

Cloning into 'badwords-classifier-train'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 221 (delta 122), reused 126 (delta 28), pack-reused 0
Receiving objects: 100% (221/221), 2.80 MiB | 2.37 MiB/s, done.
Resolving deltas: 100% (122/122), done.
/content/badwords-classifier-train
     |████████████████████████████████| 264 kB 12.4 MB/s 
     |████████████████████████████████| 282 kB 67.0 MB/s 
     |████████████████████████████████| 2.8 MB 66.4 MB/s 
     |████████████████████████████████| 919 kB 60.9 MB/s 
     |████████████████████████████████| 1.2 MB 52.9 MB/s 
     |████████████████████████████████| 302 kB 74.8 MB/s 
     |████████████████████████████████| 243 kB 74.2 MB/s 
     |████████████████████████████████| 119 kB 81.9 MB/s 
     |████████████████████████████████| 50 kB 8.9 MB/s 
     |████████████████████████████████| 3.3 MB 70.6 MB/s 
     |██████████████████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path_to_drive = "/content/drive/MyDrive/Colab Notebooks"

Mounted at /content/drive


In [ ]:
!pip install emoji
# !pip install soynlp

     |████████████████████████████████| 184 kB 13.3 MB/s 
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186469 sha256=d162d14abe7c1973ce4ce19e311d3f09d9fb990b46da52a73e2f1f49738cf79d
  Stored in directory: /root/.cache/pip/wheels/e4/61/e7/2fc1ac8f306848fc66c6c013ab511f0a39ef4b1825b11363b2
Successfully built emoji


# 시드 통일하기

In [ ]:
import torch
from transformers import set_seed


set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# 모델 불러오기

In [ ]:
# 만약 ForSequenceClassification이 붙은 모델을 사용할 경우
model_config = {
    "num_labels": 3,
    "id2label": {0: 0, 1: 1, 2: 2},
    "label2id": {0: 0, 1: 1, 2: 2}
}

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification # DistilBertForSequenceClassification
# from tokenization_kobert import KoBertTokenizer

model =AutoModelForSequenceClassification.from_pretrained("beomi/KcELECTRA-base", **model_config) # DistilBertForSequenceClassification.from_pretrained("monologg/distilkobert", **model_config)
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base", model_max_length=512) # KoBertTokenizer.from_pretrained("monologg/kobert", model_max_length=512)

# model.save_pretrained("./model")
tokenizer.tokenize("안녕하세요. 오늘 날씨는 비가 계속 내리내요.")

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.bias', 'classifier

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

['안녕', '##하세요', '.', '오늘', '날씨', '##는', '비', '##가', '계속', '내리', '##내요', '.']

# 하이퍼 파라미터 설정

In [ ]:
num_classes = 3

# 원하는 대로 고쳐서 사용
batch_size = 32
lr = 5e-06
epochs = 150

In [ ]:
# 옵티마이저와 스케줄러
# 원하는 대로 고쳐서 사용

# from torch.optim import AdamW
# from transformers import AdamW, get_linear_schedule_with_warmup


# LENGTH_OF_TRAIN_DATA = 40242
# num_training_steps = ((LENGTH_OF_TRAIN_DATA - 1) // batch_size + 1) * epochs
# optimizer = AdamW(model.parameters(), lr=lr)
# scheduler = get_linear_schedule_with_warmup(optimizer, int(num_training_steps * 0.1), num_training_steps)

# 만약 이 스케줄러를 사용할 경우,
# optimizer.step() 바로 다음에
# scheduler.step()을 호출해야 함

# f1 score 계산하기

In [ ]:
# 1. torchmetrics 사용
# requirements에 넣어놨으니 따로 설치할 필요 없음

# from torchmetrics import F1


# f1_score = F1(num_classes=num_classes)

# # 검증 단계에서 사용
# for inputs, labels in val_loader:
#     # 대충 코드
#     output = model(inputs)  # 대충 아웃풋
#     pred = torch.argmax(output, dim=1)
#     batch_f1 = f1_score(pred, labels)
#     print(batch_f1)

# f1 = f1_score.compute()
# print(f1)

In [ ]:
# 2. datasets 사용
# requirements에 넣어놨으니 따로 설치할 필요 없음
# 3번 사이킷런 사용법 쓰십시오 이건 좋지 않음

# from datasets import load_metric


# f1_score = load_metric("f1")

# for inputs, labels in val_loader:
#     # 대충 코드
#     output = model(inputs)
#     pred = torch.argmax(output, dim=1)
#     f1_score.add_batch(predictions=pred, references=labels)

# f1 = f1_score.compute()
# print(f1)

In [ ]:
# 3. sklearn 사용
# from sklearn.metrics import accuracy_score, f1_score


# def compute_metrics(p):
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)

#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     f1 = f1_score(y_true=labels, y_pred=pred)

#     return {"accuracy": accuracy, "f1": f1}

# **훈련**

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")

    return {"accuracy": accuracy, "f1": f1}

In [ ]:
from data_loader import get_data_loaders


train_data, val_data, test_data = get_data_loaders(tokenizer, return_loader=False)
# train_data = train_data.remove_columns("token_type_ids")
# val_data = val_data.remove_columns("token_type_ids")
# test_data = test_data.remove_columns("token_type_ids")

Using custom data configuration default-00927e6a6e0dcd8b


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-00927e6a6e0dcd8b/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/40242 [00:00<?, ?ex/s]

  0%|          | 0/4472 [00:00<?, ?ex/s]

  0%|          | 0/5000 [00:00<?, ?ex/s]

In [ ]:
model_name = "KcELECTRA"
save_path = f"{path_to_drive}/{model_name}"

In [ ]:
from transformers import TrainingArguments, EarlyStoppingCallback

from trainer import ImbalancedSamplerTrainer
 

# optimizer = AdamW(model.parameters(), lr =1e-4, eps = 1e-8)
# num_train_epochs = 150
# scheduler = get_polynomial_decay_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_epochs)

args = TrainingArguments(
    output_dir=save_path,
    overwrite_output_dir=True,
    learning_rate=1e-04,
    logging_dir=save_path + "/log", 
    lr_scheduler_type="linear", # (:obj:`str` or :class:`~transformers.SchedulerType`, `optional`, defaults to :obj:`"linear"`):The scheduler type to use. See the documentation of :class:`~transformers.SchedulerType` for all possible values.
    evaluation_strategy="steps",
    save_total_limit = 1,       # Only last 1 models are saved(best model). Older ones are deleted. ref= https://stackoverflow.com/questions/62525680/save-only-best-weights-with-huggingface-transformers
    load_best_model_at_end=True,  
    num_train_epochs=150,       # total number of training epochs
    seed=42,
    fp16=True,
    metric_for_best_model="eval_",
    per_device_train_batch_size=16,     # batch size per device during training
    per_device_eval_batch_size=16,      # batch size for evaluation
    greater_is_better=True,
)

trainer = ImbalancedSamplerTrainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    # tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=15)],
)

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 40242
  Num Epochs = 150
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 377400


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.250800,0.612285,0.845259,0.649226
1000,0.231800,0.362214,0.882379,0.714316
1500,0.221100,0.456099,0.863819,0.704003
2000,0.201600,0.275978,0.910778,0.755462
2500,0.159900,0.418178,0.903175,0.736301
3000,0.158000,0.378885,0.913238,0.749869
3500,0.143800,0.502506,0.879696,0.715529
4000,0.127100,0.385613,0.906753,0.743632
4500,0.113200,0.363262,0.914580,0.751179
5000,0.109000,0.442938,0.902057,0.729177


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1314: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
***** Running Evaluation *****
  Num examples = 4472
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/KcELECTRA/checkpoint-500
Configuration saved in /content/drive/MyDrive/Colab Notebooks/KcELECTRA/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/KcELECTRA/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4472
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/KcELECTRA/checkpoint-1000
Configuration saved in /content/drive/MyDrive/Colab Notebooks/KcELECTRA

TrainOutput(global_step=17000, training_loss=0.09605530323701747, metrics={'train_runtime': 18868.7671, 'train_samples_per_second': 319.91, 'train_steps_per_second': 20.001, 'total_flos': 7.154474809492685e+16, 'train_loss': 0.09605530323701747, 'epoch': 6.76})

In [ ]:
trainer.save_model(output_dir=save_path)

Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/KcELECTRA
Configuration saved in /content/drive/MyDrive/Colab Notebooks/KcELECTRA/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/KcELECTRA/pytorch_model.bin


# **모델 테스트**

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.176
cache size	: 39424 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	:

In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer


model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/KcELECTRA")
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base", model_max_length=512) # "monologg/kobert", "beomi/KcELECTRA-base"

loading configuration file /content/drive/MyDrive/Colab Notebooks/KcELECTRA/config.json
Model config ElectraConfig {
  "_name_or_path": "beomi/KcELECTRA-base",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": 0,
    "1": 1,
    "2": 2
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0": 0,
    "1": 1,
    "2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "tokenizer_class": "BertTokenizer",
  "torch_dtype": "float3

In [ ]:
# 테스트 데이터

test_df = pd.read_csv("data/test.csv")
test_data = test_df["content"]
test_label = test_df["labels"]

speed_test_data = test_data.head(50).tolist()

In [ ]:
model.to("cpu")
model.eval()
print()

In [ ]:
from data_loader import get_data_loaders


# _, _, test_dataset = get_data_loaders(tokenizer, return_loader=False, use_imbalanced=False, device="cuda") # DistilKoBERT Dataset

_, _, test_loader = get_data_loaders(tokenizer, use_imbalanced=False, device="cuda")

Using custom data configuration default-00927e6a6e0dcd8b
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-00927e6a6e0dcd8b/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-00927e6a6e0dcd8b/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-5bac84e6ee9ab0f7.arrow


  0%|          | 0/4472 [00:00<?, ?ex/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-00927e6a6e0dcd8b/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-b99d9a0761310a9a.arrow


In [ ]:
from torchmetrics import F1
from tqdm.auto import tqdm


f1_score = F1(num_classes=3, average="macro").cuda()
# f1
model.cuda()
model.eval()

pred_array = []
labels_data = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        labels = batch.pop("labels")
        output = model(**batch)
        logits = output.logits
        pred = torch.argmax(logits, dim=1)
        f1_score(pred, labels)
        pred_list = pred.tolist()
        pred_array.append(pred_list[0])
        labels_list = labels.tolist()
        labels_data.append(labels_list[0])

score = f1_score.compute()
print(score.item())

  0%|          | 0/157 [00:00<?, ?it/s]

0.9106473922729492
